# Megatron 进行预测

## CD

定位到工作目录，根据具体情况决定哦，不一定是下面的命令

In [1]:
%cd ..

/home/Public/Megatron-LM


## Importings

In [2]:
import copy
import csv
import json
import os
import random
import sys
import time
from contextlib import closing
from types import SimpleNamespace

import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm

import mpu
from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer
from pretrain_gpt2 import get_masks_and_position_ids
from predict_gpt2 import initialize_distributed, prepare_tokenizer, set_random_seed, setup_model, get_token_stream

## Args

In [3]:
args = SimpleNamespace(
    # Model arguments
    num_layers=12,
    hidden_size=768,
    num_attention_heads=12,
    max_position_embeddings=1024,
    vocab_size=None,
    make_vocab_size_divisible_by=128,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    # Train/valid/test data arguments.
    seq_length=512,
    model_parallel_size=1,
    tokenizer_model_type='bert-large-uncased',
    tokenizer_type=SentencePieceTokenizer,
    tokenizer_path="./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model",
    cache_dir=None,
    # Training arguments.
    load='./checkpoints/gpt2-117m-xinli001_qa.finetune/',
    seed=1234,
    checkpoint_activations=None,
    checkpoint_num_layers=1,
    finetune=None,
    no_load_optim=None,
    no_load_rng=None,
    resume_dataloader=None,
    fp16=True,
    hysteresis=2,
    loss_scale=None,
    loss_scale_window=1000,
    min_scale=1,
    distributed_backend='nccl',
    DDP_impl='local',
    local_rank=None,
    reset_position_ids=None,
    reset_attention_mask=None,
    eod_mask_loss=None, 
    # Text generate arguments.
    recompute=None,
    greedy=False,
    top_p=0.0,
    top_k=0,
    temperature=1.0,
    out_seq_length=128,
)

In [4]:
args.cuda = torch.cuda.is_available()
args.rank = int(os.getenv('RANK', '0'))
args.world_size = int(os.getenv("WORLD_SIZE", '1'))

if os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'):
    # We are using (OpenMPI) mpirun for launching distributed data parallel processes
    local_rank = int(os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'))
    local_size = int(os.getenv('OMPI_COMM_WORLD_LOCAL_SIZE'))

    # Possibly running with Slurm
    num_nodes = int(os.getenv('SLURM_JOB_NUM_NODES', '1'))
    nodeid = int(os.getenv('SLURM_NODEID', '0'))

    args.local_rank = local_rank
    args.rank = nodeid*local_size + local_rank
    args.world_size = num_nodes*local_size

args.model_parallel_size = min(args.model_parallel_size, args.world_size)
if args.rank == 0:
    print('using world size: {} and model-parallel size: {} '.format(
        args.world_size, args.model_parallel_size))

args.dynamic_loss_scale = False
if args.loss_scale is None:
    args.dynamic_loss_scale = True
    if args.rank == 0:
        print(' > using dynamic loss scaling')

# The args fp32_* or fp16_* meant to be active when the
# args fp16 is set. So the default behavior should all
# be false.
if not args.fp16:
    args.fp32_embedding = False
    args.fp32_tokentypes = False
    args.fp32_layernorm = False


using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling


## Init

In [5]:
%%time

# Disable CuDNN.
torch.backends.cudnn.enabled = False

# Pytorch distributed.
initialize_distributed(args)

# Random seeds for reproducability.
set_random_seed(args.seed)

# get the tokenizer
tokenizer = prepare_tokenizer(args)

# Model, optimizer, and learning rate.
model = setup_model(args)

args.device = torch.cuda.current_device()

# setting default batch size to 1
args.batch_size = 1

assert mpu.get_model_parallel_rank() == 0

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
prepare tokenizer done
building GPT2 model ...
 > number of parameters on model parallel rank 0: 110516736
global rank 0 is loading checkpoint ./checkpoints/gpt2-117m-xinli001_qa.finetune/iter_0035000/mp_rank_00/model_optim_rng.pt
  successfully loaded ./checkpoints/gpt2-117m-xinli001_qa.finetune/iter_0035000/mp_rank_00/model_optim_rng.pt
CPU times: user 5.13 s, sys: 1.73 s, total: 6.87 s
Wall time: 6.88 s


## Inference functions

In [14]:
def infer_generative(contex_text, model, tokenizer):
    contex_text = contex_text.strip()
    context_tokens = tokenizer.EncodeAsIds(contex_text).tokenization
    context_length = len(context_tokens)

    if context_length >= args.seq_length//2:
        print(
            f"\nContext length {context_length}"
            "\nPlease give smaller context (half of the sequence length)!",
            file=sys.stderr
        )

    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)
    for output_tokens, _ in token_stream:
        ids = output_tokens.cpu().numpy().tolist()[0]
        s = tokenizer.DecodeIds([ids[-1]])
        yield s

def infer_next_token_generative(context_tokens, model):
    context_length = len(context_tokens)

    if context_length >= args.seq_length//2:
        print(
            f"\nContext length {context_length}"
            "\nPlease give smaller context (half of the sequence length)!",
            file=sys.stderr
        )

    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)
    for output_tokens, _ in token_stream:
        ids = output_tokens.cpu().numpy().tolist()[0]
        next_token = ids[-1]
        yield next_token

## Infer texts

In [29]:
input_texts = [
    (
        '最近晚上睡不着觉，但很困，总觉得不会睡觉？',
        '最近一段时间，总是晚上睡不着觉，但是却很困，没次睡觉心理都在不自觉的想着该如何去睡觉，每次想着想着就完全清醒了，失眠特别严重，感觉身体快要透支了，很无助，还连累家人一起跟着担心'
    ),
    (
        '大一，谈了将近一年的女友突然提出分手我该怎么办？',
        ' 某一天晚上她突然给我打电话说我不是她喜欢的那种类型，在电话中她多次提到自己的性格不好，说“我所看到的都是加了“滤镜”的效果，其实她并没有我想得那么好，今后不准备结婚”。（她以前谈过一次恋爱，对于那场恋爱她一直都铭记在心）接完电话我很迷茫和困惑，因为之前她愿意跟我分享她的生活中的不如意和困难，突然的转变让我措手不及。同时我也很自责，没有准确的表达出自己的意思和情感，在与她的对话中我常常感到无力和无助。我到底该怎么办？我真的很爱她，想和她度过一生。'
    )
]


In [33]:
n_gen = 3

for title, text in input_texts:
    txt = f'{title}<sep>{text}<|endoftext|>'
    context_tokens = tokenizer.EncodeAsIds(txt).tokenization
    print(title)
    print(text)
    print()
    for i in range(n_gen):
        s_pred = ''
        for s in infer_generative(txt, model, tokenizer):
            if '<|endoftext|>' in s:
                s_pred = ''
            else:
                s_pred += s
        print(f'{i}): {s_pred}')
        print()
    print()
    print('=' * 100)
    print()
    

最近晚上睡不着觉，但很困，总觉得不会睡觉？
最近一段时间，总是晚上睡不着觉，但是却很困，没次睡觉心理都在不自觉的想着该如何去睡觉，每次想着想着就完全清醒了，失眠特别严重，感觉身体快要透支了，很无助，还连累家人一起跟着担心

0): 焦虑的原因已经有大概,焦虑症有情绪低落,却没有自残的行为,焦虑,过度关注自身的原因,化跑线比较差,并且过度关注自身的变化,是想发生什么事情来进展。如果出现泛化,是不是惊恐发作,导致焦虑、抑郁。 <eos>

1): 焦虑情绪息都会让我们措手不及,从而影响你的学习,生活,可以暂时自己调节 <eos>

2): 药物治疗,只有两条路才能碰到你是母亲的症状。但是不知道你指向谁吐的行为呢?看起来你有些社交焦虑,有人可以帮你安心的缓解。一步步入正轨上,跑步是跑步的,一方面是很多人都会有挑战,但一方面了解客观条件,之后就可以决定好了。 <eos>



大一，谈了将近一年的女友突然提出分手我该怎么办？
 某一天晚上她突然给我打电话说我不是她喜欢的那种类型，在电话中她多次提到自己的性格不好，说“我所看到的都是加了“滤镜”的效果，其实她并没有我想得那么好，今后不准备结婚”。（她以前谈过一次恋爱，对于那场恋爱她一直都铭记在心）接完电话我很迷茫和困惑，因为之前她愿意跟我分享她的生活中的不如意和困难，突然的转变让我措手不及。同时我也很自责，没有准确的表达出自己的意思和情感，在与她的对话中我常常感到无力和无助。我到底该怎么办？我真的很爱她，想和她度过一生。

0): 你好!困扰是时候思想和想法在作祟,看到你就不再神秘了!当你认为两人互相吸引的时候,你不敢来主动去动挡他,敢继续做朋友的关心,让自己远一些,这样让我不知道要如何去爱她。 <eos>

1): 呵呵,看完你的文字我感受到你是一个很空洞的人,面对这样的情况,有一段不太容易渡过这段伤痛。回忆是这段经历中回忆和过去经历的体验,似乎成了你成长中的一段经历,你对前任的态度和前任的态度,似乎在情感上遇到了了一些疑惑。接纳自己,才是真正的前任由过去的去面对和处理。 <eos>

2): 看完这个问题,看的出来,没什么大不了的,性格决定中,适应新的环境,是要适应一下就改变一下,如果找不到原因,可以选择咨询。 <eos>



